In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, log_loss
import matplotlib.pyplot as plt
import seaborn as sns

# Thư viện cho Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset # Để tạo Dataset object cho Hugging Face Trainer
import torch # Cần cho Hugging Face Transformers nếu dùng backend PyTorch

In [ ]:
df_g2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/cleaned_data.csv')
df_g2 = df_g2.sample(frac=1, random_state=42).reset_index(drop=True) # Xáo trộn

In [ ]:
X_g2 = df_g2['text']
y_g2 = df_g2['Topic']

In [ ]:
test_size_ratio_g2 = 0.15
validation_size_ratio_g2 = 0.15
validation_relative_ratio_g2 = validation_size_ratio_g2 / (1 - test_size_ratio_g2)

X_temp_g2, X_test_text, y_temp_g2, y_test_text = train_test_split(
    X_g2, y_g2, test_size=test_size_ratio_g2, stratify=y_g2, random_state=42
)
X_train_text, X_val_text, y_train_text, y_val_text = train_test_split(
    X_temp_g2, y_temp_g2, test_size=validation_relative_ratio_g2, stratify=y_temp_g2, random_state=42
)

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_text)
y_val_encoded = label_encoder.transform(y_val_text)
y_test_encoded = label_encoder.transform(y_test_text) # Sẽ dùng ở Giai đoạn 4
class_names_original = label_encoder.classes_
num_classes = len(class_names_original)

print(f"Số lượng lớp: {num_classes}")
print(f"Kích thước X_train_text: {X_train_text.shape}, y_train_encoded: {y_train_encoded.shape}")
print(f"Kích thước X_val_text: {X_val_text.shape}, y_val_encoded: {y_val_encoded.shape}")
print("-" * 50)

Số lượng lớp: 41
Kích thước X_train_text: (11201,), y_train_encoded: (11201,)
Kích thước X_val_text: (2401,), y_val_encoded: (2401,)
--------------------------------------------------


In [ ]:
# Hàm vẽ Confusion Matrix (giữ nguyên)
def plot_confusion_matrix_custom(y_true_encoded, y_pred_encoded, original_classes, model_name):
    cm = confusion_matrix(y_true_encoded, y_pred_encoded)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=original_classes, yticklabels=original_classes)
    plt.title(f'Confusion Matrix - {model_name} (Validation Set)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

# Hàm vẽ biểu đồ training history cho Keras
def plot_keras_history(history, model_name):
    plt.figure(figsize=(12, 4))
    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
# Danh sách để lưu kết quả của các mô hình học sâu
deep_learning_results = []

In [ ]:
# --- Mô hình 4: Fine-tuning PhoBERT (Hugging Face Transformers) ---
print("\n--- Bắt đầu xây dựng và huấn luyện mô hình PhoBERT ---")

# 4.1. Chuẩn bị dữ liệu cho PhoBERT
MODEL_NAME_PHOBERT = "vinai/phobert-base"
MAX_LEN_PHOBERT = 128 # PhoBERT có giới hạn input là 256, nhưng 128 thường đủ cho tin tức ngắn

# Tokenizer của PhoBERT
phobert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_PHOBERT)

# Hàm để tokenize dữ liệu
def tokenize_function_phobert(examples):
    return phobert_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LEN_PHOBERT)

# Tạo Dataset objects từ pandas DataFrame
train_dataset_hf = Dataset.from_pandas(pd.DataFrame({'text': X_train_text, 'label': y_train_encoded}))
val_dataset_hf = Dataset.from_pandas(pd.DataFrame({'text': X_val_text, 'label': y_val_encoded}))

# Tokenize datasets
train_dataset_tokenized = train_dataset_hf.map(tokenize_function_phobert, batched=True)
val_dataset_tokenized = val_dataset_hf.map(tokenize_function_phobert, batched=True)

# Loại bỏ cột 'text' không cần thiết sau khi tokenize
train_dataset_tokenized = train_dataset_tokenized.remove_columns(["text"])
val_dataset_tokenized = val_dataset_tokenized.remove_columns(["text"])
# Đặt định dạng để trả về PyTorch tensors
train_dataset_tokenized.set_format("torch")
val_dataset_tokenized.set_format("torch")


--- Bắt đầu xây dựng và huấn luyện mô hình PhoBERT ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Map:   0%|          | 0/11201 [00:00<?, ? examples/s]

Map:   0%|          | 0/2401 [00:00<?, ? examples/s]

In [ ]:
# 4.2. Tải mô hình PhoBERT và định nghĩa Training Arguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

model_phobert = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME_PHOBERT,
    num_labels=num_classes
).to(device)


# Định nghĩa hàm tính metrics cho Trainer
def compute_metrics_hf(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    # Có thể thêm các metrics khác như F1, precision, recall nếu muốn
    return {"accuracy": acc}

BATCH_SIZE_PHOBERT = 8 # Giảm batch size nếu gặp lỗi Out of Memory (OOM), đặc biệt khi không có GPU mạnh
EPOCHS_PHOBERT = 1 # Giữ ở mức rất thấp để chạy nhanh. Trong thực tế cần 2-5 epochs.
LEARNING_RATE_PHOBERT = 2e-5

training_args = TrainingArguments(
    output_dir="./phobert_results",
    num_train_epochs=EPOCHS_PHOBERT,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs"
)


trainer = Trainer(
    model=model_phobert,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=val_dataset_tokenized,
    compute_metrics=compute_metrics_hf,
    tokenizer=phobert_tokenizer # Truyền tokenizer để padding đúng cách khi cần
)

Sử dụng thiết bị: cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-32ae3b439e0d>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 4.3. Huấn luyện mô hình PhoBERT
print("\nBắt đầu huấn luyện PhoBERT...")
try:
    trainer.train()
    print("Huấn luyện PhoBERT hoàn tất.")

    # 4.4. Đánh giá mô hình PhoBERT trên tập Validation
    print("\nĐánh giá PhoBERT trên tập Validation...")
    eval_results = trainer.evaluate()
    acc_phobert = eval_results['eval_accuracy']

    # Lấy dự đoán xác suất để tính log_loss
    predictions_output = trainer.predict(val_dataset_tokenized)
    y_val_pred_proba_phobert = tf.nn.softmax(predictions_output.predictions, axis=1).numpy() # Chuyển logits sang probabilities
    y_val_pred_phobert_encoded = np.argmax(y_val_pred_proba_phobert, axis=1)
    ll_phobert = log_loss(y_val_encoded, y_val_pred_proba_phobert)

    deep_learning_results.append({'model': 'PhoBERT', 'accuracy': acc_phobert, 'log_loss': ll_phobert})

    print("\nKết quả PhoBERT trên tập Validation:")
    print(f"Accuracy: {acc_phobert:.4f}")
    print(f"Log Loss: {ll_phobert:.4f}")
    print("Classification Report (nhãn gốc):\n",
          classification_report(y_val_encoded, y_val_pred_phobert_encoded,
                                target_names=class_names_original, zero_division=0))

    # Vẽ Confusion Matrix
    plot_confusion_matrix_custom(y_val_encoded, y_val_pred_phobert_encoded,
                           classes=class_names_original, model_name="PhoBERT")

    # Vẽ biểu đồ training/validation loss và accuracy (từ log_history của Trainer)
    # Trainer log history có thể phức tạp hơn Keras history.
    # Đây là một cách đơn giản hóa để trích xuất:
    train_log = [log for log in trainer.state.log_history if 'loss' in log and 'eval_loss' not in log]
    eval_log = [log for log in trainer.state.log_history if 'eval_loss' in log]

    if train_log and eval_log:
        epochs = [log['epoch'] for log in train_log]
        train_loss = [log['loss'] for log in train_log]
        # eval_accuracy = [log['eval_accuracy'] for log in eval_log] # Cần đảm bảo eval_accuracy có trong log
        eval_loss = [log['eval_loss'] for log in eval_log]

        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        # plt.plot(epochs, train_accuracy, label='Train Accuracy') # Trainer không log train_accuracy mặc định
        # plt.plot(epochs, eval_accuracy, label='Validation Accuracy')
        # plt.title('PhoBERT Accuracy')
        # plt.xlabel('Epoch')
        # plt.ylabel('Accuracy')
        # plt.legend()
        # Vì Trainer không log train accuracy mỗi step/epoch, chúng ta chỉ vẽ loss
        plt.plot([log['epoch'] for log in train_log], [log['loss'] for log in train_log], label='Train Loss')
        plt.plot([log['epoch'] for log in eval_log], [log['eval_loss'] for log in eval_log], label='Validation Loss')
        plt.title('PhoBERT Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print("Không đủ log để vẽ biểu đồ training history cho PhoBERT.")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("\nLỖI: CUDA out of memory! Hãy thử giảm BATCH_SIZE_PHOBERT và/hoặc MAX_LEN_PHOBERT.")
        print("Nếu không có GPU hoặc GPU yếu, huấn luyện Transformer có thể rất khó khăn.")
    else:
        raise e
except Exception as e:
    print(f"Đã có lỗi xảy ra trong quá trình huấn luyện PhoBERT: {e}")
    print("Bỏ qua phần PhoBERT và tiếp tục...")


print("-" * 50)
print("Giai đoạn 3B hoàn tất.")


Bắt đầu huấn luyện PhoBERT...


Step,Training Loss
500,0.703200


Huấn luyện PhoBERT hoàn tất.

Đánh giá PhoBERT trên tập Validation...



Kết quả PhoBERT trên tập Validation:
Accuracy: 0.8626
Log Loss: 0.5543
Classification Report (nhãn gốc):
                          precision    recall  f1-score   support

                     AI       0.88      0.78      0.83        64
                Bóng đá       0.84      0.97      0.90        66
          Chuyển đổi số       0.71      0.57      0.63        63
              Chân dung       0.83      0.90      0.87        50
              Chính trị       0.93      0.94      0.93        66
            Chứng khoán       0.86      0.97      0.91        59
               Các bệnh       0.00      0.00      0.00         2
  Các môn thể thao khác       0.86      0.97      0.91        66
                 Du học       0.89      0.92      0.90        51
                Du lịch       0.88      0.85      0.86        26
               Dân sinh       0.81      0.79      0.80        66
                  Ebank       0.85      0.94      0.89        66
             Giao thông       0.88      0.98   